In [9]:
import SimpleITK as sitk
import numpy as np

def check_image_properties(image, name="Image"):
    """
    Print image properties for debugging
    """
    print(f"\n{name} Properties:")
    print(f"  Size: {image.GetSize()}")
    print(f"  Spacing: {image.GetSpacing()}")
    print(f"  Origin: {image.GetOrigin()}")
    print(f"  Physical size: {[s*sp for s, sp in zip(image.GetSize(), image.GetSpacing())]}")

def simple_rigid_registration(fixed_image, moving_image, output_path=None, 
                             original_cb_size=[512, 512, 90]):
    """
    Simplified rigid registration that handles size differences more robustly
    
    Args:
        fixed_image_path: Path to CT image e.g. (512x512x512)
        moving_image_path: Path to Cone Beam image e.g. (128x128x128)
        output_path: Optional path to save registered image
        original_cb_size: Original cone beam dimensions before downsampling
    
    Returns:
        registered_image: The registered cone beam image
        transform: The computed transformation
    """

    
    check_image_properties(fixed_image, "Fixed (CT)")
    check_image_properties(moving_image, "Moving (Cone Beam - small)")
    
    # Simple approach: Scale up the cone beam using the size ratio
    current_size = list(moving_image.GetSize())
    scale_factors = [original_cb_size[i] / current_size[i] for i in range(3)]
    
    print(f"Scale factors: {scale_factors}")
    
    # Create a new image with target size and adjusted spacing
    current_spacing = list(moving_image.GetSpacing())
    # Keep spacing roughly the same physical scale
    new_spacing = [current_spacing[i] / scale_factors[i] for i in range(3)]
    
    # Ensure no zero or negative spacing
    new_spacing = [max(0.1, abs(s)) for s in new_spacing]
    
    print(f"New spacing: {new_spacing}")
    
    # Resample to larger size
    scaled_moving = sitk.Resample(
        moving_image,
        original_cb_size,
        sitk.Transform(),
        sitk.sitkLinear,
        moving_image.GetOrigin(),
        new_spacing,
        moving_image.GetDirection(),
        0.0,
        moving_image.GetPixelID()
    )
    
    check_image_properties(scaled_moving, "Scaled Moving")
    
    # Registration setup
    registration_method = sitk.ImageRegistrationMethod()
    
    registration_method.SetMetricAsCorrelation()
        
    # Set optimizer
    registration_method.SetOptimizerAsRegularStepGradientDescent(
        learningRate=0.3,
        minStep=1e-9,
        numberOfIterations=200,
        maximumStepSizeInPhysicalUnits=1.0
    )
    registration_method.SetOptimizerScalesFromPhysicalShift()
    
    # Set initial transform
    initial_transform = sitk.CenteredTransformInitializer(
        fixed_image,
        scaled_moving,
        sitk.Euler3DTransform(),
        sitk.CenteredTransformInitializerFilter.MOMENTS
    )
    registration_method.SetInitialTransform(initial_transform, inPlace=False)
    registration_method.SetInterpolator(sitk.sitkLinear)

    # Add multi-resolution strategy for better convergence
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors=[8, 4, 2, 1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[4, 2, 1, 0])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()
    
    
    # Execute registration
    print("Starting registration...")
    final_transform = registration_method.Execute(fixed_image, scaled_moving)
    
    print(f"Final metric value: {registration_method.GetMetricValue()}")
    
    # Apply transform
    registered_image = sitk.Resample(
        scaled_moving,
        fixed_image,
        final_transform,
        sitk.sitkLinear,
        0.0,
        scaled_moving.GetPixelID()
    )

    if output_path != None:
        # Write images to disk
        print("Saving images")
        sitk.WriteImage(fixed_image, output_path + f"scan-{output_path.split("/")[-2]}-fixed_image.nii.gz")
        sitk.WriteImage(moving_image, output_path + f"scan-{output_path.split("/")[-2]}-moving_image.nii.gz")
        sitk.WriteImage(scaled_moving, output_path + f"scan-{output_path.split("/")[-2]}-scaled_moving.nii.gz")
        sitk.WriteImage(registered_image, output_path + f"scan-{output_path.split("/")[-2]}-registered_img.nii.gz")
    
    return registered_image, final_transform, scaled_moving



### Pelvis (child)

In [ ]:
ct_path = ""
cb_path = ""
output_path = ""
varianCBCT_path = ""
inputCBCT_path = ""


# Read images
print("Loading images...")
fixed_image = sitk.ReadImage(ct_path, sitk.sitkFloat32)

# Flip the CT image along the Z-axis (superior-inferior)
fixed_image_flipped = sitk.Flip(fixed_image, [False, False, True], flipAboutOrigin=True)

moving_image = sitk.ReadImage(cb_path, sitk.sitkFloat32)
#Flip Generated CBCT
moving_image_flipped = sitk.Flip(moving_image, flipAxes=(True, False, False), flipAboutOrigin=True)


#Load and flip input CBCT
input_moving_image = sitk.ReadImage(inputCBCT_path, sitk.sitkFloat32)
#Flip Generated CBCT
input_moving_image_flipped = sitk.Flip(input_moving_image, flipAxes=(True, False, False), flipAboutOrigin=True)


varian_moving_image = sitk.ReadImage(varianCBCT_path, sitk.sitkFloat32)

varian_size = varian_moving_image.GetSize()
print("Varian image size: ", varian_size)


Loading images...
Varian image size:  (512, 512, 90)


In [24]:
inputRegisteredImg, inputTransform, scaledInputMoving = simple_rigid_registration(fixed_image_flipped, input_moving_image_flipped, output_path+"input/", varian_size)
print(inputTransform)


Fixed (CT) Properties:
  Size: (512, 512, 563)
  Spacing: (0.9765625, 0.9765625, 1.0)
  Origin: (249.51171875, 72.01171875, -193.79998779296875)
  Physical size: [500.0, 500.0, 563.0]

Moving (Cone Beam - small) Properties:
  Size: (512, 512, 93)
  Spacing: (0.5111907124519348, 0.5111907124519348, 1.9897245168685913)
  Origin: (261.7296447753906, -0.5111907124519348, 1.9897245168685913)
  Physical size: [261.7296447753906, 261.7296447753906, 185.044380068779]
Scale factors: [1.0, 1.0, 0.967741935483871]
New spacing: [0.5111907124519348, 0.5111907124519348, 2.0560486674308778]

Scaled Moving Properties:
  Size: (512, 512, 90)
  Spacing: (0.5111907124519348, 0.5111907124519348, 2.0560486674308778)
  Origin: (261.7296447753906, -0.5111907124519348, 1.9897245168685913)
  Physical size: [261.7296447753906, 261.7296447753906, 185.044380068779]
Starting registration...
Final metric value: -0.1452445426951073
Saving images
itk::simple::CompositeTransform
 CompositeTransform (0x115c980)
   RTT

In [11]:

varianRegisteredImg, varianTransform, scaledVarianMoving = simple_rigid_registration(fixed_image_flipped, varian_moving_image, output_path+"varian/", varian_size)
print(varianTransform)



Fixed (CT) Properties:
  Size: (512, 512, 563)
  Spacing: (0.9765625, 0.9765625, 1.0)
  Origin: (249.51171875, 72.01171875, -193.79998779296875)
  Physical size: [500.0, 500.0, 563.0]

Moving (Cone Beam - small) Properties:
  Size: (512, 512, 90)
  Spacing: (0.5111907124519348, 0.5111907124519348, 1.9897245168685913)
  Origin: (130.60923767089844, 130.60923767089844, 85.55815887451172)
  Physical size: [261.7296447753906, 261.7296447753906, 179.07520651817322]
Scale factors: [1.0, 1.0, 1.0]
New spacing: [0.5111907124519348, 0.5111907124519348, 1.9897245168685913]

Scaled Moving Properties:
  Size: (512, 512, 90)
  Spacing: (0.5111907124519348, 0.5111907124519348, 1.9897245168685913)
  Origin: (130.60923767089844, 130.60923767089844, 85.55815887451172)
  Physical size: [261.7296447753906, 261.7296447753906, 179.07520651817322]
Starting registration...
Final metric value: -0.8265088641937333
Saving images
itk::simple::CompositeTransform
 CompositeTransform (0x173c480)
   RTTI typeinfo: 

In [12]:
# Perform registration
registered_img, transform, scaled_moving = simple_rigid_registration(fixed_image_flipped, moving_image_flipped, output_path, varian_size)

# Print transformation parameters
print("\nTransformation parameters:")
print(transform)



Fixed (CT) Properties:
  Size: (512, 512, 563)
  Spacing: (0.9765625, 0.9765625, 1.0)
  Origin: (249.51171875, 72.01171875, -193.79998779296875)
  Physical size: [500.0, 500.0, 563.0]

Moving (Cone Beam - small) Properties:
  Size: (128, 128, 128)
  Spacing: (2.0447628498077393, 2.0447628498077393, 1.4456591606140137)
  Origin: (260.1960726380348, -0.5111907124519348, 1.9897245168685913)
  Physical size: [261.7296447753906, 261.7296447753906, 185.04437255859375]
Scale factors: [4.0, 4.0, 0.703125]
New spacing: [0.5111907124519348, 0.5111907124519348, 2.056048583984375]

Scaled Moving Properties:
  Size: (512, 512, 90)
  Spacing: (0.5111907124519348, 0.5111907124519348, 2.056048583984375)
  Origin: (260.1960726380348, -0.5111907124519348, 1.9897245168685913)
  Physical size: [261.7296447753906, 261.7296447753906, 185.04437255859378]
Starting registration...
Final metric value: -0.053344673380313214
Saving images

Transformation parameters:
itk::simple::CompositeTransform
 CompositeTran

### Head scan

In [ ]:
ct_path = ""
cb_path = ""
output_path = ""
varianCBCT_path = ""
inputCBCT_path = ""



# Read images
print("Loading images...")
fixed_image = sitk.ReadImage(ct_path, sitk.sitkFloat32)

# Flip the CT image along the Z-axis (superior-inferior)
fixed_image_flipped = sitk.Flip(fixed_image, [False, False, True], flipAboutOrigin=True)

moving_image = sitk.ReadImage(cb_path, sitk.sitkFloat32)
#Flip Generated CBCT
moving_image_flipped = sitk.Flip(moving_image, flipAxes=(True, False, False), flipAboutOrigin=True)


#Load and flip input CBCT
input_moving_image = sitk.ReadImage(inputCBCT_path, sitk.sitkFloat32)
#Flip Generated CBCT
input_moving_image_flipped = sitk.Flip(input_moving_image, flipAxes=(True, False, False), flipAboutOrigin=True)


varian_moving_image = sitk.ReadImage(varianCBCT_path, sitk.sitkFloat32)

varian_size = varian_moving_image.GetSize()
print("Varian image size: ", varian_size)

Loading images...
Varian image size:  (512, 512, 93)


In [26]:
inputRegisteredImg, inputTransform, scaledInputMoving = simple_rigid_registration(fixed_image_flipped, input_moving_image_flipped, output_path+"input/", varian_size)
print(inputTransform)


Fixed (CT) Properties:
  Size: (512, 512, 204)
  Spacing: (1.5234375, 1.5234375, 2.0)
  Origin: (389.23828125, 174.73828125, -226.0)
  Physical size: [780.0, 780.0, 408.0]

Moving (Cone Beam - small) Properties:
  Size: (512, 512, 93)
  Spacing: (0.5111907124519348, 0.5111907124519348, 1.9897245168685913)
  Origin: (261.7296447753906, -0.5111907124519348, 1.9897245168685913)
  Physical size: [261.7296447753906, 261.7296447753906, 185.044380068779]
Scale factors: [1.0, 1.0, 1.0]
New spacing: [0.5111907124519348, 0.5111907124519348, 1.9897245168685913]

Scaled Moving Properties:
  Size: (512, 512, 93)
  Spacing: (0.5111907124519348, 0.5111907124519348, 1.9897245168685913)
  Origin: (261.7296447753906, -0.5111907124519348, 1.9897245168685913)
  Physical size: [261.7296447753906, 261.7296447753906, 185.044380068779]
Starting registration...
Final metric value: -0.6306553844707264
Saving images
itk::simple::CompositeTransform
 CompositeTransform (0x12b7300)
   RTTI typeinfo:   itk::Composi

In [13]:
varianRegisteredImg, varianTransform, scaledVarianMoving = simple_rigid_registration(fixed_image_flipped, varian_moving_image, output_path+"varian/", varian_size)
print(varianTransform)


Loading images...
Varian image size:  (512, 512, 93)

Fixed (CT) Properties:
  Size: (512, 512, 204)
  Spacing: (1.5234375, 1.5234375, 2.0)
  Origin: (389.23828125, 174.73828125, -226.0)
  Physical size: [780.0, 780.0, 408.0]

Moving (Cone Beam - small) Properties:
  Size: (512, 512, 93)
  Spacing: (0.5111907124519348, 0.5111907124519348, 1.9897245168685913)
  Origin: (130.60923767089844, 130.60923767089844, 91.52732849121094)
  Physical size: [261.7296447753906, 261.7296447753906, 185.044380068779]
Scale factors: [1.0, 1.0, 1.0]
New spacing: [0.5111907124519348, 0.5111907124519348, 1.9897245168685913]

Scaled Moving Properties:
  Size: (512, 512, 93)
  Spacing: (0.5111907124519348, 0.5111907124519348, 1.9897245168685913)
  Origin: (130.60923767089844, 130.60923767089844, 91.52732849121094)
  Physical size: [261.7296447753906, 261.7296447753906, 185.044380068779]
Starting registration...
Final metric value: -0.7522746508722393
Saving images
itk::simple::CompositeTransform
 CompositeTra

In [14]:

# Perform registration
registered_img, transform, scaled_moving = simple_rigid_registration(fixed_image_flipped, moving_image_flipped, output_path, varian_size)

# Print transformation parameters
print("\nTransformation parameters:")
print(transform)



Fixed (CT) Properties:
  Size: (512, 512, 204)
  Spacing: (1.5234375, 1.5234375, 2.0)
  Origin: (389.23828125, 174.73828125, -226.0)
  Physical size: [780.0, 780.0, 408.0]

Moving (Cone Beam - small) Properties:
  Size: (128, 128, 128)
  Spacing: (2.0447628498077393, 2.0447628498077393, 1.4456591606140137)
  Origin: (260.1960726380348, -0.5111907124519348, 1.9897245168685913)
  Physical size: [261.7296447753906, 261.7296447753906, 185.04437255859375]
Scale factors: [4.0, 4.0, 0.7265625]
New spacing: [0.5111907124519348, 0.5111907124519348, 1.9897244361139113]

Scaled Moving Properties:
  Size: (512, 512, 93)
  Spacing: (0.5111907124519348, 0.5111907124519348, 1.9897244361139113)
  Origin: (260.1960726380348, -0.5111907124519348, 1.9897245168685913)
  Physical size: [261.7296447753906, 261.7296447753906, 185.04437255859375]
Starting registration...
Final metric value: -0.4959804588514615
Saving images

Transformation parameters:
itk::simple::CompositeTransform
 CompositeTransform (0x11

### Thorax scan

In [ ]:
ct_path = ""
cb_path = ""
output_path = ""
varianCBCT_path = ""
inputCBCT_path = ""


# Read images
print("Loading images...")
fixed_image = sitk.ReadImage(ct_path, sitk.sitkFloat32)

# Flip the CT image along the Z-axis (superior-inferior)
fixed_image_flipped = sitk.Flip(fixed_image, [False, False, True], flipAboutOrigin=True)

moving_image = sitk.ReadImage(cb_path, sitk.sitkFloat32)
#Flip Generated CBCT
moving_image_flipped = sitk.Flip(moving_image, flipAxes=(True, False, False), flipAboutOrigin=True)



#Load and flip input CBCT
input_moving_image = sitk.ReadImage(inputCBCT_path, sitk.sitkFloat32)
#Flip Generated CBCT
input_moving_image_flipped = sitk.Flip(input_moving_image, flipAxes=(True, False, False), flipAboutOrigin=True)



varian_moving_image = sitk.ReadImage(varianCBCT_path, sitk.sitkFloat32)

varian_size = varian_moving_image.GetSize()
print("Varian image size: ", varian_size)


Loading images...
Varian image size:  (512, 512, 88)


In [28]:

inputRegisteredImg, inputTransform, scaledInputMoving = simple_rigid_registration(fixed_image_flipped, input_moving_image_flipped, output_path+"input/", varian_size)
print(inputTransform)


Fixed (CT) Properties:
  Size: (512, 512, 187)
  Spacing: (1.3671875, 1.3671875, 2.0)
  Origin: (349.31640625, 114.81640625, -293.9000244140625)
  Physical size: [700.0, 700.0, 374.0]

Moving (Cone Beam - small) Properties:
  Size: (512, 512, 87)
  Spacing: (0.9080203771591187, 0.9080203771591187, 2.0113463401794434)
  Origin: (464.90643310546875, -0.9080203771591187, 2.0113463401794434)
  Physical size: [464.90643310546875, 464.90643310546875, 174.98713159561157]
Scale factors: [1.0, 1.0, 1.0114942528735633]
New spacing: [0.9080203771591187, 0.9080203771591187, 1.9884901317683132]

Scaled Moving Properties:
  Size: (512, 512, 88)
  Spacing: (0.9080203771591187, 0.9080203771591187, 1.9884901317683132)
  Origin: (464.90643310546875, -0.9080203771591187, 2.0113463401794434)
  Physical size: [464.90643310546875, 464.90643310546875, 174.98713159561157]
Starting registration...
Final metric value: -0.3834571147994282
Saving images
itk::simple::CompositeTransform
 CompositeTransform (0x1dca

In [ ]:
varianRegisteredImg, varianTransform, scaledVarianMoving = simple_rigid_registration(fixed_image_flipped, varian_moving_image, output_path+"varian/", varian_size)
print(varianTransform)


Loading images...
Varian image size:  (512, 512, 88)

Fixed (CT) Properties:
  Size: (512, 512, 187)
  Spacing: (1.3671875, 1.3671875, 2.0)
  Origin: (349.31640625, 114.81640625, -293.9000244140625)
  Physical size: [700.0, 700.0, 374.0]

Moving (Cone Beam - small) Properties:
  Size: (512, 512, 88)
  Spacing: (0.9080203771591187, 0.9080203771591187, 1.988490104675293)
  Origin: (231.99920654296875, 231.99920654296875, 86.49932098388672)
  Physical size: [464.90643310546875, 464.90643310546875, 174.98712921142578]
Scale factors: [1.0, 1.0, 1.0]
New spacing: [0.9080203771591187, 0.9080203771591187, 1.988490104675293]

Scaled Moving Properties:
  Size: (512, 512, 88)
  Spacing: (0.9080203771591187, 0.9080203771591187, 1.988490104675293)
  Origin: (231.99920654296875, 231.99920654296875, 86.49932098388672)
  Physical size: [464.90643310546875, 464.90643310546875, 174.98712921142578]
Starting registration...
Final metric value: -0.8652532210511396
Saving images
itk::simple::CompositeTransf

In [16]:
# Perform registration
registered_img, transform, scaled_moving = simple_rigid_registration(fixed_image_flipped, moving_image_flipped, output_path, varian_size)

# Print transformation parameters
print("\nTransformation parameters:")
print(transform)


Fixed (CT) Properties:
  Size: (512, 512, 187)
  Spacing: (1.3671875, 1.3671875, 2.0)
  Origin: (349.31640625, 114.81640625, -293.9000244140625)
  Physical size: [700.0, 700.0, 374.0]

Moving (Cone Beam - small) Properties:
  Size: (128, 128, 128)
  Spacing: (3.6320815086364746, 3.6320815086364746, 1.3670870065689087)
  Origin: (462.1823719739914, -0.9080203771591187, 2.0113463401794434)
  Physical size: [464.90643310546875, 464.90643310546875, 174.9871368408203]
Scale factors: [4.0, 4.0, 0.6875]
New spacing: [0.9080203771591187, 0.9080203771591187, 1.988490191372958]

Scaled Moving Properties:
  Size: (512, 512, 88)
  Spacing: (0.9080203771591187, 0.9080203771591187, 1.988490191372958)
  Origin: (462.1823719739914, -0.9080203771591187, 2.0113463401794434)
  Physical size: [464.90643310546875, 464.90643310546875, 174.9871368408203]
Starting registration...
Final metric value: -0.307162649941052
Saving images

Transformation parameters:
itk::simple::CompositeTransform
 CompositeTransfo